In [1]:
import json
import pandas as pd
import numpy as np
import xnetwork as xnet
from tqdm.auto import tqdm
from collections import Counter
from itertools import combinations
from collections import defaultdict, namedtuple
from multiprocessing import Pool

In [105]:
import json
import igraph as ig
import WOSRaw as wos
import xnetwork as xnet
from collections import defaultdict

acm_ami = 'ACS Appl. Mater. Interfaces'
adv_f_mater = 'Adv. Funct. Mater.'

valid_journals = [
    'acs nano',
    'adv. mater.',
    'chem. mat.', 
    'j. am. chem. soc.', 
    'j. mat. chem. b', 
    'j. mater. chem. a', 
    'j. mater. chem. c', 
    'langmuir',
    'macromolecules',
    'nano lett.',
    'nat. mater.',
    'nat. nanotechnol.'
]


def get_citnet(current_journal):
    
    papers_by_journal = defaultdict(lambda:[])
    names = []
    titles = []
    years = []
    edges = []
    file = open('toy_chemb_chema_natmat.txt').readlines()
    for line in file:
        entry = json.loads(line)
        item_id = entry['UID']
        item_data = {'UID': item_id, 'data': entry}
        sourcesData = wos.utilities.getSources(item_data)
        journal = wos.utilities.getSourceISOAbbreviation(sourcesData).lower()
        
        publicationInfo = wos.utilities.getPublicationInfo(item_data)
        year = wos.utilities.getPublicationYear(publicationInfo)
        year = int(year)
        
        if journal != current_journal:
            continue
        if year > 2020:
            continue
            
        title = wos.utilities.getTitle(item_data)
        names.append(item_id)
        titles.append(title)
        years.append(int(year))
        
            
        referencesData = wos.utilities.getReferences(item_data)
        ref_uids = wos.utilities.getReferencesUIDs(referencesData)
        for current_uid in ref_uids:
            edges.append((item_id, current_uid))
        
    valid_edges = []
    names_set = set(names)
    for edge in edges:
        if edge[0] in  names_set and edge[1] in names_set:
            valid_edges.append(edge)
    
    
    g = ig.Graph(directed=True)
    g.add_vertices(len(names))
    g.vs['name'] = names
    g.vs['title'] = titles
    g.vs['year'] = years
    g.add_edges(valid_edges)
    return g

for journal in valid_journals:
    print(journal)
    g = get_citnet(journal)
    xnet.igraph2xnet(g, '%s_completedata_170423_allbut.xnet' % journal)

j. mat. chem. b
j. mater. chem. a
nat. mater.
